In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b7
import torch.nn as nn
import torch.optim as optim
from pathlib import Path

In [7]:
transform = transforms.Compose([
    transforms.Resize(224),  # EfficientNet-B7의 입력 크기에 맞게 조정
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

datadir = Path().home() / "dataset"

trainset = torchvision.datasets.CIFAR10(root=datadir, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=datadir, train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
# EfficientNet-B7 모델 로드 및 수정
model = efficientnet_b7(pretrained=False)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)  # 출력층을 CIFAR-10의 클래스 수에 맞게 조정

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.245
[1,   400] loss: 2.036


KeyboardInterrupt: 

In [ ]:
# 테스트
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test images: {100 * correct / total:.2f}%')